# Лабораторная работа №3

## Тема: Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей.

## Выполнил: Фонин Максим Алексеевич ИУ5-61Б

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import * 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style ="ticks")


from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, Normalizer

In [2]:
df = pd.read_csv('AMD_specs.csv')
df.head()

,model,launchDate,numCores,numThreads,baseClock,maxboostClock,allCoreBoostSpeed,L1Cache,L2Cache,L3Cache,...,graphicsCores,graphicsFreq,unlocked,platform,cpuSocket,socketCount,family,line,graphicsModel,thermalSolution
0,AMD Ryzen™ 5 6600H,2022.0,6.0,12.0,3.3,4.5,NaN,384.0,3.0,16.0,...,6.0,1900.0,No,Laptop,FP7,NaN,AMD Ryzen™ Processors,AMD Ryzen™ 5 Mobile Processors with Radeon™ Gr...,AMD Radeon™ 660M,NaN
1,AMD Ryzen™ 5 6600HS​,2022.0,6.0,12.0,3.3,4.5,NaN,384.0,3.0,16.0,...,6.0,1900.0,No,Laptop,FP7,NaN,AMD Ryzen™ Processors,AMD Ryzen™ 5 Mobile Processors with Radeon™ Gr...,AMD Radeon™ 660M,NaN
2,AMD Ryzen™ 5 6600U,2022.0,6.0,12.0,2.9,4.5,NaN,384.0,3.0,16.0,...,6.0,1900.0,No,Laptop,FP7,NaN,AMD Ryzen™ Processors,AMD Ryzen™ 5 Mobile Processors with Radeon™ Gr...,AMD Radeon™ 660M,NaN
3,AMD Ryzen™ 5 PRO 6650H,2022.0,6.0,12.0,3.3,4.5,NaN,384.0,3.0,16.0,...,6.0,1900.0,No,Laptop,NaN,NaN,AMD Ryzen™ PRO Processors,AMD Ryzen™ 5 PRO 6000 Series Mobile Processors,AMD Radeon™ 660M,NaN
4,AMD Ryzen™ 5 PRO 6650HS,2022.0,6.0,12.0,3.3,4.5,NaN,384.0,3.0,16.0,...,6.0,1900.0,No,Laptop,NaN,NaN,AMD Ryzen™ PRO Processors,AMD Ryzen™ 5 PRO 6000 Series Mobile Processors,AMD Radeon™ 660M,NaN


In [3]:
# удаляем ненужные строки
df.drop(['model','allCoreBoostSpeed','L1Cache','L2Cache','L3Cache','unlocked','platform','cpuSocket','socketCount','family','line', 'graphicsModel', 'thermalSolution'], axis=1, inplace=True)
df.head()

,launchDate,numCores,numThreads,baseClock,maxboostClock,processTech,PCIeVersion,PCIeLanes,defaultTDP,configTDP,tjMax,sysMemType,sysMemSpecs,memChannels,perSocketMemBW,graphicsCores,graphicsFreq
0,2022.0,6.0,12.0,3.3,4.5,6.0,4.0,NaN,45.0,NaN,95.0,DDR5,NaN,4.0,NaN,6.0,1900.0
1,2022.0,6.0,12.0,3.3,4.5,6.0,4.0,NaN,35.0,NaN,95.0,DDR5,NaN,4.0,NaN,6.0,1900.0
2,2022.0,6.0,12.0,2.9,4.5,6.0,4.0,NaN,15.0,15-28,95.0,DDR5,NaN,4.0,NaN,6.0,1900.0
3,2022.0,6.0,12.0,3.3,4.5,6.0,4.0,NaN,45.0,NaN,95.0,DDR5,NaN,4.0,NaN,6.0,1900.0
4,2022.0,6.0,12.0,3.3,4.5,6.0,4.0,NaN,35.0,NaN,95.0,DDR5,NaN,4.0,NaN,6.0,1900.0


In [4]:
# удаляем ненужные строки
df.drop(['launchDate', 'PCIeLanes','configTDP','sysMemType','sysMemSpecs','perSocketMemBW'], axis=1, inplace=True)
df.head()

,numCores,numThreads,baseClock,maxboostClock,processTech,PCIeVersion,defaultTDP,tjMax,memChannels,graphicsCores,graphicsFreq
0,6.0,12.0,3.3,4.5,6.0,4.0,45.0,95.0,4.0,6.0,1900.0
1,6.0,12.0,3.3,4.5,6.0,4.0,35.0,95.0,4.0,6.0,1900.0
2,6.0,12.0,2.9,4.5,6.0,4.0,15.0,95.0,4.0,6.0,1900.0
3,6.0,12.0,3.3,4.5,6.0,4.0,45.0,95.0,4.0,6.0,1900.0
4,6.0,12.0,3.3,4.5,6.0,4.0,35.0,95.0,4.0,6.0,1900.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   numCores       577 non-null    float64
 1   numThreads     522 non-null    float64
 2   baseClock      582 non-null    float64
 3   maxboostClock  501 non-null    float64
 4   processTech    480 non-null    float64
 5   PCIeVersion    324 non-null    float64
 6   defaultTDP     574 non-null    float64
 7   tjMax          447 non-null    float64
 8   memChannels    402 non-null    float64
 9   graphicsCores  254 non-null    float64
 10  graphicsFreq   296 non-null    float64
dtypes: float64(11)
memory usage: 50.1 KB


In [6]:
df.isna().sum()

numCores           5
numThreads        60
baseClock          0
maxboostClock     81
processTech      102
PCIeVersion      258
defaultTDP         8
tjMax            135
memChannels      180
graphicsCores    328
graphicsFreq     286
dtype: int64

In [7]:
columns_to_replace = ['numThreads']
for column in columns_to_replace:
    df[column] = df[column].fillna(1)

In [8]:
df.isna().sum()

numCores           5
numThreads         0
baseClock          0
maxboostClock     81
processTech      102
PCIeVersion      258
defaultTDP         8
tjMax            135
memChannels      180
graphicsCores    328
graphicsFreq     286
dtype: int64

In [9]:
df.dropna(subset=['numCores', 'defaultTDP'],inplace = True, axis = 0)
df.isna().sum()

numCores           0
numThreads         0
baseClock          0
maxboostClock     78
processTech       99
PCIeVersion      246
defaultTDP         0
tjMax            132
memChannels      175
graphicsCores    319
graphicsFreq     280
dtype: int64

In [10]:
columns_to_replace = ['maxboostClock']
for column in columns_to_replace:
    df[column] = df[column].fillna(df['baseClock'])
df.isna().sum()

numCores           0
numThreads         0
baseClock          0
maxboostClock      0
processTech       99
PCIeVersion      246
defaultTDP         0
tjMax            132
memChannels      175
graphicsCores    319
graphicsFreq     280
dtype: int64

In [11]:
# удаляем ненужные строки
df.drop(['processTech', 'PCIeVersion','tjMax','memChannels','graphicsCores','graphicsFreq'], axis=1, inplace=True)
df.isna().sum()

numCores         0
numThreads       0
baseClock        0
maxboostClock    0
defaultTDP       0
dtype: int64

In [12]:
df.head()

,numCores,numThreads,baseClock,maxboostClock,defaultTDP
0,6.0,12.0,3.3,4.5,45.0
1,6.0,12.0,3.3,4.5,35.0
2,6.0,12.0,2.9,4.5,15.0
3,6.0,12.0,3.3,4.5,45.0
4,6.0,12.0,3.3,4.5,35.0


In [13]:
y = df['baseClock']  #Наименования признаков
X = df.drop('baseClock', axis=1) # Значения признаков

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state= 45)
# random_state позволяет задавать базовое значение для генератора случайных чисел, чтобы сделать выборку неслучайной 

In [15]:
# Размер тестовой выборки
X_test.shape, y_test.shape

((143, 4), (143,))

In [16]:
np.unique(y_train)

array([0.2 , 1.  , 1.2 , 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 , 1.65,
       1.7 , 1.8 , 1.9 , 2.  , 2.05, 2.1 , 2.2 , 2.25, 2.3 , 2.4 , 2.45,
       2.5 , 2.6 , 2.65, 2.7 , 2.75, 2.8 , 2.85, 2.9 , 3.  , 3.1 , 3.2 ,
       3.3 , 3.4 , 3.5 , 3.6 , 3.7 , 3.8 , 3.9 , 4.  , 4.1 , 4.2 , 4.4 ,
       4.7 ])

In [17]:
np.unique(y_test)

array([1.3 , 1.55, 1.6 , 1.8 , 1.9 , 2.  , 2.1 , 2.2 , 2.3 , 2.35, 2.4 ,
       2.5 , 2.6 , 2.7 , 2.8 , 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.2 , 3.3 ,
       3.4 , 3.5 , 3.6 , 3.7 , 3.8 , 3.9 , 4.  , 4.2 ])

In [18]:
#Масштабирование данных
scaler = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_train.columns)
X_train.describe()

,numCores,numThreads,maxboostClock,defaultTDP
count,426.000000,426.000000,426.000000,426.000000
mean,0.110064,0.110070,0.641696,0.267351
std,0.190134,0.192056,0.194812,0.244092
min,0.000000,0.000000,0.000000,0.000000
25%,0.032258,0.023622,0.550000,0.112480
50%,0.032258,0.055118,0.675000,0.221156
75%,0.096774,0.118110,0.775000,0.329832
max,1.000000,1.000000,1.000000,1.000000


In [19]:
def test_model(model):
    print("mean_absolute_error:",
          mean_absolute_error(y_test, model.predict(X_test)))
    print("mean_squared_error:",
          mean_squared_error(y_test, model.predict(X_test)))
    print("median_absolute_error:",
          median_absolute_error(y_test, model.predict(X_test)))
    print("r2_score:",
          r2_score(y_test, model.predict(X_test)))

In [20]:
reg_10 = KNeighborsRegressor(n_neighbors=10)
reg_10.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [21]:
test_model(reg_10)

mean_absolute_error: 0.2056993006993007
mean_squared_error: 0.07422150349650349
median_absolute_error: 0.17999999999999972
r2_score: 0.8230377667686481
